# Getting Started with Neo4j for Spatial Analysis

Introduction & background on Neo4j and graph databases...

## Launch Neo4j locally using Docker

### Create a persistent data volume

To get started, you'll need to create a persistent data volume that Docker can use to store your database's files.  Creating a Docker volume allows you to persist any data loaded into the database, even if you have to stop, start, or re-run the Docker container housing your database.  Run the following from the command line to create a Docker volume for you Neo4j database:

```bash
docker volume create neo4j_volume
```

To check that the volume was created successfully, run the following from your command line:
```bash
docker volume ls
```

You should see a volume called `neo4j_volume` listed in the results.

### Download database plugins

Neo4j has a number of community-supported plugins that provide useful utilities for loading data, running advanced graph algorithms, and managing spatial data.  Download the latest release of each of these plugins from their respective GitHub pages:

- Awesome Procedures for Neo4j (APOC): 
[[Download link](https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases)]
[[GitHub main page](https://github.com/neo4j-contrib/neo4j-apoc-procedures)]
[[Documentation](https://neo4j-contrib.github.io/neo4j-apoc-procedures/)]

- Efficient Graph Algorithms for Neo4j: 
[[Download link](https://github.com/neo4j-contrib/neo4j-graph-algorithms/releases)]
[[GitHub main page](https://github.com/neo4j-contrib/neo4j-graph-algorithms)]
[[Documentation](https://neo4j.com/docs/graph-algorithms/current/)]

- Neo4j Spatial: 
[[Download link](https://github.com/neo4j-contrib/spatial/releases)]
[[GitHub main page](https://github.com/neo4j-contrib/spatial)]
[[User Guide](https://neo4j-contrib.github.io/spatial/)]


Each of these plugins downloads as a .jar file.  Take these .jar files and move them to a folder on your computer where you can find them easily.  For this example, we'll assume you're saving them to your user folder inside of a folder called 'neo4j_plugins':

/Users/your_username/neo4j_plugins

### Launch the Docker container

Now you're finally ready to launch Neo4j using Docker!  Run the following from your command line.  Be sure to replace the file path in brackets (<>) below with the actual path to your neo4j_plugins folder (and remove the brackets):

```bash
docker run -d \
--name neo4j \
-p 7474:7474 -p 7687:7687 \
--mount source=neo4j_volume,target=/data \
-v </Users/your_username/neo4j_plugins>:/plugins \
-e NEO4J_apoc_export_file_enabled=true \
-e NEO4J_apoc_import_file_enabled=true \
-e NEO4J_apoc_import_file_use__neo4j__config=true \
-e NEO4J_dbms_security_procedures_unrestricted=apoc.\\\*,algo.\\\*,spatial.\\\* \
-e NEO4J_dbms_security_allow__csv__import__from__file__urls=true \
neo4j:3.4
```

What does this command do?  Breaking it down, here's what each argument means:

- **-d** : runs the container in "detached" mode, so that it keeps running in the background and will not shut off if you close out of your console window
- **--name** : the name Docker will give to your container; if you don't specify a name here, Docker will give your container a randomly-generated name
- **-p** : these are port mappings, indicating that Docker should forward information going in and out of port 7474 from the Neo4j container to port 7474 on your local machine; the same goes for port 7687
- **--mount** : takes the persistent volume named `neo4j_volume` and mounts it into the /data folder inside of the container; this is where the database's data and settings will get stored
- **-e**: environment variables that get passed to the database's configuration file on startup; these variables are required to make sure plugins will run correctly
- At the very end of the command, you'll notice we list **neo4j:3.4** as the final argument.  This specifies the Docker image and version to run inside of the container, and will download and launch Neo4j version 3.4 (the most current version as of this writing).  If newer versions are available, you can specify `neo4j:latest` to get the most recent version of Neo4j.

To check that the container is running, execute the following command in the command line:

```docker container ls```

You should see something like this, indicating that the container is successfully running:

```
CONTAINER ID    IMAGE        COMMAND                CREATED         STATUS          PORTS
blahblahblah    neo4j:3.4    "/sbin/tini -g -- /d…" X seconds ago   Up X seconds    0.0.0.0:7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp
```

<hr>

## Connect to the database

Neo4j has a web-based access interface that you can view using a web browser.  Launch your web browser of choice and navigate to:

```http://localhost:7474/browser```

If the container is successfully initialized, you should be able to see a login interface in your web browser.  It will prompt you for a username and password.  Enter ```neo4j``` as both your username and password:

![Neo4j browser launch page](img_neo4j/neo4j_browser_launch.png)

Then, reset the default admit password to a new password of your choice when prompted:

![Neo4j browser launch page password setup](img_neo4j/neo4j_browser_launch_2.png)

<hr>

## Load data into the database

Now, it's almost time to load in some data!  Because it is a graph database, Neo4j requires a little consideration of how to structure the data into **nodes** and their **relationships** (sometimes called "edges").  Nodes are the _things_ we want to represent.  For this example, the node types will be: "tweets", "users", "places", and "hashtags".  Relationships are the _linkages_ we want to represent between nodes.  For this example, we'll define relationships such as: "tweeted by", "located in", "mentions", etc.  Both nodes and relationships can contain attributes that describe them in further detail.  Together, nodes and edges and the properties that describe them represent the fundamental building blocks of a "property graph" data structure.  

Here's a sketch of the data model we'll use when loading tweets into Neo4j:
<img src="img_neo4j/neo4j_data_model.png" alt="Neo4j tweet data model" style="width: 600px; height: 500px"/>
Don't worry if the data structure doesn't feel totally comprehensive.  The big advantage of a graph database like Neo4j is that it's easy to add additional relationships between nodes at any time via simple queries if you want to add additional structure to your data later on.

### Create indexes

Prior to loading in any data, let's prepare the database by adding some indexes.  Adding indexes is a critical step, because the presence of indexes can really help improve load time.  We also want to do our best not to load duplicate data when adding data to the database, and indexes can help with this.  When loading in data, we will ask Neo4j to check for any duplicates of tweets, users, places, or hashtags that may be present prior to loading in new records.  If it encounters a tweet that is already present in the database, Neo4j can skip over this tweet and avoid loading a redundant record.  In order to check for duplicates, however, Neo4j first needs to search through all of the existing records on each record load to see what's already present in the database.  This search and insert process goes _much_ more quickly if data is pre-sorted using indexes.  

_What happens if we don't add indexes before loading in the data?_  Here is an example of 40 files being loaded into Neo4j installed on a home server.  In this scenario, the first 20 files were loaded prior to the addition of indexes, and the last 20 files were loaded after the addition of indexes:  

![visualization of load time before and after adding indexes](img_neo4j/neo4j_load_time_indexes.png)

Some of the load time depicted above is due to network latency and processing effort required in the database, but much of the load time can be attributed to the relative efficiency or ineffeciency due to the presence or absence of indexes.  For the first 20 files, you can see that the load time increases at a linear rate, with each file taking consistently longer to load.  For the last 20 files, you can see that the load time immediately drops after the addition of the indexes and stays more consistently flat, regardless of how many additional files we load.

To create an index, run the following command in the web admin console for your Neo4j instance.  Copy the command into the console, and then click the "play" button to execute the command: 

```cypher
CREATE INDEX ON :Tweet(tweet_id)
```

![Neo4j create index call](img_neo4j/neo4j_create_index.png)

The above command created an index on tweet_id, which is the unique identifier we'll be using to load tweets and check for duplicates.  Next, create an index on user_id, place_id, and hashtag_id to function in a similar manner.  Execute these commands one by one in the Neo4j web admin console:

```cypher
CREATE INDEX ON :User(user_id)
CREATE INDEX ON :Place(place_id)
CREATE INDEX ON :Hashtag(hashtag_id)
```

Finally, add a spatial index for the centroids of the Twitter Places that are present in our tweets.  This will enable some basic spatial query functionality after the data is loaded:

```cypher
CREATE INDEX ON :Place(centroid)
```

To check that the indexes have been successfully created, run the following:
```cypher
CALL db.indexes
```

![Neo4j results for CALL db.indexes](img_neo4j/neo4j_indexes.png)

### Execute load scripts

Now it's time to load in the data.  The demo data comes in the form of .txt files that have been split into chunks of ~5000 tweets per file.  Each line of each file is formatted as a JSON object representing a single tweet, and each tweet is separated by a newline within the .txt files.  To make loading easier, this repository contains a pre-baked script you can use to load data into Neo4j.  The script cleans the tweet data's geographic components and also coerces the data to conform to the basic data model described above as it is loaded into Neo4j.  Run the following code to import the script:

In [1]:
import Clean_Load_Scripts as cleanNLoad
import secrets

Next, define a variable called `data_folder` that points to the location of the demo data on your computer.  Also define a variable called `logs_folder` that points to the path where you want to store log files about the load.  This folder can be located directly inside of your data folder, or it can be an entirely separate path.  The load script will keep track of files that need to be loaded, load time for each file, error counts, and some other diagnostic data for each file as it loads.

In [4]:
data_folder = '/Users/linkalis/Desktop/twitter_data/data_small_5000_split/'
logs_folder = '/Users/linkalis/Desktop/twitter_data/data_small_5000_split/logs/'

Note that it will ~30 seconds to load each file, so be ready grab a coffee and be patient.  Because you have already added indexes, and because the data is being loaded to your localhost machine and doesn't have to travel across a network, this should keep the load time relatively flat.  This is about what you should expect:

![Neo4j load times in Docker running on localhost](img_neo4j/neo4j_load_time_docker_localhost.png)

Now, initialize the extractor to kick off the load logs and prep the files for load:

In [6]:
extractor = cleanNLoad.Extractor(data_folder, logs_folder, initialize=True)

Finally, run the `while` loop below that actually does the work of cleaning and loading each file into the database. Before running the `while` loop, remember to replace the `username` and `password` fields with the username and password you set up in the steps above--and be sure to add quotes('') around them.

Also, if the while loop is interrupted for some reason during the load, don't panic!  The logs folder contains a `files_to_load.txt` file that keeps track of all the files that still need to be loaded into the database.  To re-start the load, simply re-load the extractor above with the argument `initialize=False`.  Then, re-run the while loop below and the load will pick up where it left off.

Ready to load?  Okay, go!

In [ ]:
while extractor.next_file_available():
    next_file_data, next_file_name = extractor.get_next_file() # read in the next file
    cleaner = cleanNLoad.Cleaner(next_file_data, next_file_name, logs_folder) # clean the data (fix bounding boxes, add centroids, etc.)
    cleaned_data = cleaner.clean_data() 
    loader = cleanNLoad.Loader(cleaned_data, next_file_name, logs_folder) # initialize the loader
    loader.get_connection("neo4j", "localhost", "7687", <username>, <password>) # create a database connection
    loader.load_data() # load the data file

<hr>


## Query the data

### Basic queries

Now that the data is loaded, let's look at how to perform a few basic queries.  Neo4j uses a query language called **Cypher**, which is specialized for graph data structures.  If you are familiar with relational query languages like SQL, you may notice some slight similarities when it comes to high-level operators.  For example, the `RETURN` clause in Cypher is like the `SELECT` clause in SQL, and the `WHERE` clause is present in both query languages.  Much of the rest of the structure is quite different, however.  A basic query in Cypher is structured using the following operators, in the following order:

```
MATCH <node/relationship pattern to find>
WITH <aggregations and calculations>
WHERE <filter clauses>
RETURN <fields/attributes you want returned in the result set>
ORDER BY <fields/attributes to sort by>
LIMIT <maximum number of results to return>
```

Now, let's look at some sample questions and how you can query for answers using Cypher.  To run the queries below, simply paste them into the Neo4j browser window's query prompt and click the "play" button to return the results.

#### Query: What are the top 5 places where people are tweeting, in order of popularity?

```Cypher
MATCH (t:Tweet)-[:LOCATED_AT]->(p:Place)
WITH p, count(t) as tweet_count
RETURN p.name, p.country_code, tweet_count 
ORDER BY tweet_count DESC
LIMIT 5
```

Notice the `MATCH` criterion above?  Cypher has a very visual way of depicting the pattern of nodes and relationships you want to detect using your query.  After the `MATCH` predicate, you can specify a specific pattern of nodes and relationships to search for in the database.  Parentheses `()` will match a node, and arrows and brackets `-[]->` will match a relationship.  Inside the parentheses, you can specify the type of node you want to search for--in this case, `(:Place)`.  You can also assign an arbitrary variable name--in this case, `p`--to use throughout the rest of the query to indicate this type of node.  Inside the brackets of a relationship, you can specify the relationship type--in this case, `[:LOCATED_AT]`.  Also be sure to point the arrows surrounding the relationship brackets in the right direction between the nodes, based on the direction of the relationship that was set up when we imported the data.

#### Query: What are the top 10 most popular hashtags?

```Cypher
MATCH (t:Tweet)-[:HASHTAGS]->(h:Hashtag) 
WITH h, count(t) as tweet_count_per_hashtag
RETURN h
ORDER BY tweet_count_per_hashtag DESC
LIMIT 10
```

In this query, we're simply returning the top 10 hashtag nodes.  Notice that, if your query returns full nodes (and not just specific node attributes) in its result, Neo4j will automatically display the nodes in a cool, clickable visual structure for you.  Hover over one of the hashtag bubbles that appears in your browser, click it, then click the "expand" icon to view all of the tweets that are associated with the hashtag.

### Spatial queries

Neo4j also offers support for some basic [spatial functions](https://neo4j.com/docs/developer-manual/current/cypher/functions/spatial/) and [spatial indexing for point data](https://neo4j.com/docs/developer-manual/current/cypher/syntax/spatial/).  In this sample data, Twitter provides coordinates that use WGS-84 as the coordinate reference system (CRS) when tagging its tweets.  When interpreting the results of spatial queries in Neo4j, the [documentation](https://neo4j.com/docs/developer-manual/current/cypher/functions/spatial/) explains that "if the points are in the WGS-84 CRS (2D), then the units of the returned distance will be meters, based on the haversine formula over a spherical earth approximation".

So, what's going on behind the scenes in Neo4j when it comes to querying spatial data?  According to the [official documentation](https://neo4j.com/docs/developer-manual/current/cypher/syntax/spatial/), "for spatial indexing, Neo4j uses space filling curves in 2D or 3D over an underlying generalized B+Tree. [...] This allows for both equality and range queries using exactly the same syntax and behaviour as for other property types."  

Earlier, we created a spatial index on the "centroid" attribute of each Twitter Place in our dataset.  Let's use this centroid data to try some spatial queries...

#### Query: Which tweets were tweeted within 100 miles of the UMN Duluth campus?

```cypher
MATCH (t:Tweet)-[:LOCATED_AT]->(p:Place)
WHERE distance(p.centroid, point({ latitude:toFloat('46.820058'), longitude:toFloat('-92.084323') })) < 160934
RETURN t, p
```

This query uses the `distance()` function in the `WHERE` clause to compute the distance between the centroid of each tweet and a specific point ([46.820058, -92.084323]) located in the middle of the Duluth campus.  The `WHERE` clause also restricts the results to include _only_ those tweets that are located at a place whose centroid is < 160,934 meters (approximately 100 miles) from the Duluth campus.

**Challenge:** Try to adapt the query above so that it returns a list of all of the place names and the count of tweets that are located in each place whose centroid falls within 100 miles of UMD.

#### Query: Which tweets were tweeted within a bounding box surrounding the Twin Cities?

```cypher
MATCH (t:Tweet)-[:LOCATED_AT]->(p:Place)
WHERE point({ latitude:toFloat('44.858683'), longitude:toFloat('-93.350877') }) < p.centroid < point({latitude:toFloat('45.072057'), longitude:toFloat('-92.956315') })
RETURN t, p
```

The query above takes advantage of the fact that points in Neo4j behave like any other data type when it comes to equality and comparison operators.  Because of how Neo4j indexes spatial points, a bounding box query can be represented as a simple range query that looks for points that fall between the lower left coordinate ([44.858683, -93.350877]) and upper right coordinate ([45.072057, -92.956315]) of a given bounding box.

#### Query: Find tweets that are located in places with place type of "poi" in the Twin Cities bounding box.  Then join to the Yelp dataset to find restaurants that are located near each of these tweets.


<span style="background:yellow">Add Yelp dataset and spatial join query</span>

### Advanced queries

#### Query: What is the shortest path between Britney Spears and Donald Trump, based on users' mentioning behavior?

Right now, the data model in our Neo4j database is set up so that `(t:Tweet)` nodes can have a `[:MENTIONS]` relationship that connects them to `(u:User)` nodes.  There is not, however, a direct user-to-user link between users who have mentioned each other, without first having identify specific tweets.  In order to answer the question above, it would be really helpful to extend the `[:MENTIONS]` relationship so that it can connect user nodes directly to other user nodes, instead of always having to go through tweets.  

Fortunately, Neo4j makes it relatively easy to adjust the data structure to add new relationships between nodes.  So, let's make a quick adjustment to the `[:MENTIONS]` relationship.  If a user creates a tweet that mentions another user, let's add an additional `[:MENTIONS]` relationship between the user and their mentionee. This is something we could have added to the original data model when importing the data, but it's not a problem to add this new relationship later on.  Run this query to add a `[:MENTIONS]` relationship between users and each of their mentionees:

```cypher
MATCH (u1:User)-[:TWEETED]->(t:Tweet)-[:MENTIONS]->(u2:User)
WHERE u1 <> u2
MERGE (u1)-[r:MENTIONS]->(u2)
```

Next, use the shortestPath() algorithm to find mention patterns between users that link Britney Spears and Donald Trump:

```cypher
MATCH path = shortestPath((u1:User)-[:MENTIONS*..10]-(u2:User))
WHERE u1.screen_name = "britneyspears"
AND u2.screen_name = "realDonaldTrump"
RETURN path
```

Note that the `[:MENTIONS*..10]` syntax above allows a variable-length path of up to 10 degrees of mention separation between Britney and Trump.  Also notice that this query will match users who mention _or were mentioned by_ other users; i.e. it doesn't force a directional constraint on the `[:MENTIONS]` relationship, and the path between users can go in either direction.  This dataset is relatively small, so it doesn't contain anywhere close to a full approximation of mentioners and their mentionees.  Because of this, it will generally only return results when there is no directional constraint on the `[:MENTIONS]` pattern we are trying to match.  In a larger dataset, you may be able to add a directional constraint (ex: `(u1:User)-[:MENTIONS*..10]->(u2:User)`) to force a specific direction for the `[:MENTION]` relationship.

#### Query: Which users appear to be the most influential, based on how many mentions they've received?

For this query, we can take advantage of a popular algorithm called PageRank that is implemented in the Neo4j graph algorithms library.  The Neo4j documentation describes the [PageRank algorithm](https://neo4j.com/docs/graph-algorithms/current/algorithms/page-rank/) as follows: "PageRank is named after Google co-founder Larry Page, and is used to rank websites in Google’s search results. It counts the number, and quality, of links to a page which determines an estimation of how important the page is. The underlying assumption is that pages of importance are more likely to receive a higher volume of links from other pages."  The algorithm operates similar to a search engine ranking; we can set up a query so that users who receive lots of mentions--and who receive mentions from other who _themselves_ receive lots of mentions--will surface as the most influential.  Run the following query to see which of the Twitter users present in our sample data appears to be the most influential, based on the quantity of their mentions and the caliber of their mentioners:

```cypher
CALL algo.pageRank.stream('User', 'MENTIONS', { iterations: 20, dampingFactor: 0.85 })
YIELD nodeId, score
MATCH (u) WHERE id(u) = nodeId
RETURN u.screen_name AS user, score
ORDER BY score DESC
```

Are you surprised by any of the results?

<span style="background:yellow">
CALL algo.closeness.stream('User', 'MENTIONS')
YIELD nodeId, centrality
MATCH (n:User) WHERE id(n) = nodeId
RETURN n.screen_name AS node, centrality
ORDER BY centrality DESC
LIMIT 20
</span>

### Query challenges

Here are some additional questions you can try to answer with Neo4j and the sample Twitter dataset.  There are often many ways to get at a justifiable answer to these questions, so be creative!

<span style="background:yellow">Add query challenge questions here</span>

## Resources

- Get Started with Cypher. [Neo4j documentation]. https://neo4j.com/docs/developer-manual/current/get-started/cypher/
- Intro to Cypher. [Neo4j documentation]. https://neo4j.com/developer/cypher-query-language/ 
- Indexes. [Neo4j documentation]. https://neo4j.com/docs/developer-manual/current/cypher/schema/index/
- Spatial functions. [Neo4j documentation]. https://neo4j.com/docs/developer-manual/current/cypher/functions/spatial/
- Spatial values. [Neo4j documentation]. https://neo4j.com/docs/developer-manual/current/cypher/syntax/spatial/
- Up and Running with Neo4j. [Lynda]. https://www.lynda.com/Neo4j-tutorials/Up-Running-Neo4j/155604-2.html
- Database Clinic: Neo4j. [Lynda]. https://www.lynda.com/Neo4j-tutorials/Database-Clinic-Neo4J/601789-2.html
- Algorithms. [Neo4j documentation]. https://neo4j.com/docs/graph-algorithms/current/algorithms/
- What's New in Neo4j 3.4: Spatial Features. https://medium.com/neo4j/whats-new-in-neo4j-spatial-features-586d69cda8d0
- Small, Nigel. A Pythonic Tour of Neo4j and PyMongo. [PyData Amsterdam 2017]. https://www.youtube.com/watch?v=Ma6lVy6x3Mg